In [1]:
import time
import os
import pandas as pd
import numpy as np
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
data_path="../../data/"
output_path="../../outputs/"

In [3]:
df=pd.read_csv(output_path+"sg_ie/positives_ready.csv")

In [4]:
df=df.assign(abstract_for_prompt=df.abstract_for_prompt.fillna(""))

df=df.assign(paper_text="Title: "+df.title_clean+"\n"+df.abstract_for_prompt)

ack_preprompt="""Identify the NLP task(s) addressed in this paper. Answer in the format [task1,...,taskn,...]"""

ack_postprompt="""The NLP tasks addressed in this paper are:\n["""

df=df.assign(task_prompt_text=ack_preprompt+"\n"+df.paper_text+"\n"+ack_postprompt)

## cost estimation

In [10]:
np.sum(df['task_prompt_text'].str.split().apply(len))

1475484

In [ ]:
$0.0600 / 1K tokens

In [11]:
df.shape

(8952, 15)

In [14]:
8952*40

358080

In [ ]:
1627921

In [15]:
(((1475484+358080)*1000/750)/1000)*0.0600

146.68511999999998

## requests

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['task_prompt_text']
    if ((i%100==0) or i<100):
        print(input_prompt)
        print("###################################")

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['task_prompt_text']
    completion = openai.Completion.create(engine="text-davinci-002", prompt=input_prompt,temperature=0,max_tokens=40)
    df.loc[i,'GPT3_response']=completion.choices[0].text
    f = open("/cluster/scratch/fgonzalez/logs_gpt3/responses_full.txt", "a")
    f.write(d['task_prompt_text'])
    f.write(completion.choices[0].text)
    f.write("################################")
    f.write("\n")
    f.close()
    if ((i%100==0) or i<100):
        print(input_prompt)
        print(completion.choices[0].text)
        print("################################")

In [18]:
df=df.assign(clean_response=df.GPT3_response.replace("\n","",regex=True))
df=df.assign(clean_response=df.clean_response.replace("(?:The|The NLP|)(?:NLP|)\stasks addressed in this paper\s?(?:are|include|):","",regex=True))
df=df.assign(clean_response=df.clean_response.replace("NLP tasks:","",regex=True))
df=df.assign(clean_response=df.clean_response.replace("[?Tt]ask\s?\d: ","",regex=True).replace("\[?[tT]ask\d\]",",",regex=True))
df=df.assign(clean_response=df.clean_response.replace("\[?\d\]",",",regex=True).replace("\d\.",",",regex=True).replace("\(?\d\)",",",regex=True).replace("task1,\s?task2,\.\.\.\]","",regex=True))
df=df.assign(clean_response=df.clean_response.replace(',,',',',regex=True).replace('  ',' ',regex=True).str.rstrip(']').str.lstrip(',').str.lstrip(' '))

df=df.assign(clean_response=df.clean_response.str.split(","))

tasks=df.loc[:,['ID','clean_response']]

tasks=tasks.explode("clean_response")

tasks=tasks.rename(columns={'clean_response':'tasks'})

tasks=tasks.assign(tasks=tasks.tasks.str.lstrip().str.rstrip())
tasks=tasks.loc[tasks.tasks!='']

In [19]:
tasks

,ID,tasks
0,lin-etal-2006-generative,discourse analysis
0,lin-etal-2006-generative,text classification
1,ghosh-etal-2020-iitp,Offensive Language Identification
1,ghosh-etal-2020-iitp,Target Categorization
2,grouin-2016-identification,identification of mentions
...,...,...
8950,Language (technology) is power: The need to be...,stance detection
8950,Language (technology) is power: The need to be...,opinion mining
8951,Gender in Danger? Evaluating Speech Translatio...,speech recognition
8951,Gender in Danger? Evaluating Speech Translatio...,speech translation


In [20]:
df.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_task_full.csv",index=False)

tasks.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_task_full_cleaned.csv",index=False)